In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003) (3).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,23:42]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')


d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,3,5,7,8,9,10,11,13,15,17], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)

d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def error(x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 8))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
        print(pp,py, r)
    return emin , emax;



for x4 in range(10,100):
                dif=3.0
                model = basic_model_3(x4)
                epochs = 500
                batch_size = 128
                #print('Epochs: ', epochs)
                #print('Batch size: ', batch_size)
                keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]



                history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=0, # Change it to 2, if wished to observe execution
                validation_data=(X_test, Y_test),
                callbacks=keras_callbacks)
                
                train_score = model.evaluate(X_train, Y_train, verbose=0)
                valid_score = model.evaluate(X_test, Y_test, verbose=0)
            
                emn , ema = error(50)
                print('errmin,errmax,x4:',emn,ema , x4)
               

Using TensorFlow backend.
W0930 11:18:12.327879 14068 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 11:18:12.398468 14068 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 11:18:12.406469 14068 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0930 11:18:12.478466 14068 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0930 11:18:12.7

[[9.848546]] [[7.97]] [[1.2710744]]
[[9.7418375]] [[10.67]] [[0.90361756]]
[[10.883328]] [[13.]] [[0.82302076]]
[[10.796194]] [[13.]] [[0.8157353]]
[[10.39638]] [[8.]] [[1.3443075]]
[[10.791868]] [[11.58]] [[0.9252246]]
[[9.713895]] [[12.]] [[0.7914138]]
[[10.911303]] [[6.46]] [[1.8212736]]
[[10.086904]] [[8.]] [[1.2998424]]
[[11.42352]] [[17.93]] [[0.6147733]]
[[11.645214]] [[13.84]] [[0.8285323]]
[[12.242482]] [[11.]] [[1.1247472]]
[[10.389589]] [[14.65]] [[0.68696463]]
[[9.976492]] [[10.27]] [[0.9682006]]
[[12.008696]] [[14.]] [[0.84635]]
[[11.759655]] [[13.]] [[0.89629215]]
[[8.704135]] [[8.45]] [[1.034296]]
[[10.672463]] [[14.]] [[0.7432456]]
[[11.009555]] [[14.]] [[0.76925576]]
[[10.106205]] [[15.]] [[0.6494416]]
[[10.340429]] [[11.91]] [[0.85560524]]
[[12.352034]] [[12.]] [[1.0321198]]
[[9.536152]] [[8.26]] [[1.1767522]]
[[9.521903]] [[8.55]] [[1.1294144]]
[[9.63228]] [[11.56]] [[0.81675667]]
[[9.720717]] [[14.]] [[0.66980845]]
[[11.427425]] [[11.]] [[1.0429142]]
[[9.920956]] [[

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[9.911775]] [[7.97]] [[1.2801983]]
[[9.773531]] [[10.67]] [[0.9069087]]
[[10.894244]] [[13.]] [[0.8239334]]
[[10.878762]] [[13.]] [[0.8226389]]
[[10.4016075]] [[8.]] [[1.3450586]]
[[10.823881]] [[11.58]] [[0.9282619]]
[[9.780939]] [[12.]] [[0.7975309]]
[[11.007333]] [[6.46]] [[1.8389913]]
[[10.064761]] [[8.]] [[1.2966611]]
[[11.506615]] [[17.93]] [[0.619693]]
[[11.686564]] [[13.84]] [[0.8317628]]
[[12.37774]] [[11.]] [[1.1383272]]
[[10.427116]] [[14.65]] [[0.689722]]
[[10.0718155]] [[10.27]] [[0.9785282]]
[[12.153642]] [[14.]] [[0.8575341]]
[[11.86505]] [[13.]] [[0.90510446]]
[[8.752221]] [[8.45]] [[1.0407856]]
[[10.658842]] [[14.]] [[0.7421946]]
[[11.205165]] [[14.]] [[0.78434914]]
[[10.190282]] [[15.]] [[0.6554643]]
[[10.405329]] [[11.91]] [[0.8615758]]
[[12.4605255]] [[12.]] [[1.0420188]]
[[9.50759]] [[8.26]] [[1.1727965]]
[[9.585375]] [[8.55]] [[1.1378661]]
[[9.699891]] [[11.56]] [[0.82318354]]
[[9.773871]] [[14.]] [[0.67390984]]
[[11.488445]] [[11.]] [[1.0490406]]
[[9.981441]] [[

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[9.437755]] [[7.97]] [[1.2117972]]
[[8.709189]] [[10.67]] [[0.79638517]]
[[10.996264]] [[13.]] [[0.83246356]]
[[10.518509]] [[13.]] [[0.7925174]]
[[10.987953]] [[8.]] [[1.4293035]]
[[9.9620285]] [[11.58]] [[0.8464922]]
[[9.546192]] [[12.]] [[0.7761124]]
[[9.332162]] [[6.46]] [[1.5299193]]
[[8.714279]] [[8.]] [[1.1026262]]
[[12.161406]] [[17.93]] [[0.658461]]
[[11.608205]] [[13.84]] [[0.825641]]
[[12.226008]] [[11.]] [[1.1230931]]
[[9.596939]] [[14.65]] [[0.62872434]]
[[9.07463]] [[10.27]] [[0.87049073]]
[[13.142863]] [[14.]] [[0.9338629]]
[[13.024534]] [[13.]] [[1.0020512]]
[[7.8498735]] [[8.45]] [[0.91901124]]
[[11.349016]] [[14.]] [[0.7954488]]
[[12.813139]] [[14.]] [[0.9084212]]
[[11.678508]] [[15.]] [[0.7620707]]
[[9.436918]] [[11.91]] [[0.77248555]]
[[11.905408]] [[12.]] [[0.9913693]]
[[9.0392685]] [[8.26]] [[1.1079319]]
[[9.289458]] [[8.55]] [[1.098463]]
[[9.874758]] [[11.56]] [[0.83980584]]
[[11.620247]] [[14.]] [[0.81637704]]
[[10.304]] [[11.]] [[0.9301204]]
[[8.595493]] [[12.

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[10.045087]] [[7.97]] [[1.2994351]]
[[10.408297]] [[10.67]] [[0.97282416]]
[[11.298927]] [[13.]] [[0.8577698]]
[[11.239679]] [[13.]] [[0.8528159]]
[[11.191374]] [[8.]] [[1.4585307]]
[[10.590514]] [[11.58]] [[0.9061208]]
[[10.889949]] [[12.]] [[0.89871794]]
[[9.99751]] [[6.46]] [[1.652677]]
[[10.122051]] [[8.]] [[1.3048923]]
[[11.903672]] [[17.93]] [[0.6432014]]
[[12.430861]] [[13.84]] [[0.88991106]]
[[12.54912]] [[11.]] [[1.155534]]
[[10.634184]] [[14.65]] [[0.7049363]]
[[9.777796]] [[10.27]] [[0.9466734]]
[[13.303395]] [[14.]] [[0.94624966]]
[[12.863955]] [[13.]] [[0.9886249]]
[[8.864955]] [[8.45]] [[1.0559992]]
[[11.471266]] [[14.]] [[0.80488163]]
[[12.899831]] [[14.]] [[0.9151104]]
[[11.462691]] [[15.]] [[0.7466111]]
[[10.465834]] [[11.91]] [[0.867142]]
[[12.615928]] [[12.]] [[1.0561978]]
[[9.615155]] [[8.26]] [[1.1876945]]
[[9.279905]] [[8.55]] [[1.097191]]
[[10.48239]] [[11.56]] [[0.8975656]]
[[11.790518]] [[14.]] [[0.8295152]]
[[10.928826]] [[11.]] [[0.9928539]]
[[9.699597]] [[1

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [ ]:
42,51,57

In [ ]:
print(d1)